In [1]:
%pip install langchain openai chromadb faiss-cpu tiktoken pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os                                                                                                                                                                                                                                                                                                                                                                              
from langchain_openai import ChatOpenAI  
from langchain_core.prompts import PromptTemplate  
from langchain.chains import LLMChain

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_path = "domain_knowledge.pdf"  # Replace with your actual file path
loader = PyPDFLoader(pdf_path)

# Extract pages
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Generate embeddings
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")  # Saves for later retrieval

C:\Users\admin\AppData\Local\Temp\ipykernel_9752\899472323.py:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************zkoA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Load the FAISS index
# vectorstore = FAISS.load_local("faiss_index", embeddings)
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Retrieve top 3 relevant chunks

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # This handles combining retrieved documents
    retriever=retriever
)

C:\Users\admin\AppData\Local\Temp\ipykernel_1464\3677377817.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)


In [ ]:
problem_statement="""Saturn Int. management wants to improve their security measures, both for their building and on site. They would like to prevent people who are not part of the company to use their car park.
Saturn Int. has decided to issue identity cards to all employees. Each card records the name, department and number of a company staff, and give them access to the company car park. Employees are asked to wear the cards while on the site.

There is a barrier and a card reader placed at the entrance to the car park. When a driver drives his car into the car park, he/she inserts his or her identity card into the card reader. The card reader then verify the card number to see if it is known to the system. If the number is recognized, the reader sends a signal to trigger the barrier to rise. The driver can then drive his/her car into the car park.

There is another barrier at the exit of the car park, which is automatically raised when a car wishes to leave the car park.

A sign at the entrance display “Full” when there are no spaces in the car park. It is only switched off when a car leaves.

There is another type of card for guests, which also permits access to the car park. The card records a number and the current date. Such cards may be sent out in advance, or collected from reception. All guest cards must be returned to reception when the visitor leaves Saturn Int."""

In [ ]:
few_shot_examples = """
Example 1:
Problem: "An online store allows users to browse products, add them to cart, and checkout."
Activity Diagram:
1. Start
2. User selects product
3. Add to cart
4. Proceed to checkout
5. Payment processing
6. Order confirmation
7. End

Example 2:
Problem: "A user logs into a banking app and transfers funds."
Activity Diagram:
1. Start
2. Enter credentials
3. Validate login
4. Navigate to transfer funds
5. Enter recipient and amount
6. Confirm transaction
7. End
"""

In [ ]:
def generate_activity_diagram(problem_statement):
    # Retrieve domain knowledge
    context = rag_chain.run(problem_statement)

    # Construct the final prompt
    prompt = f"""
    You are an expert in UML diagrams. Your task is to generate an activity diagram for the given problem.

    Context: {context}

    Here are some examples:
    {few_shot_examples}

    Now, generate an activity diagram for the following problem:
    Problem: "{problem_statement}"
    Activity Diagram:
    """
    return prompt


In [ ]:
def get_activity_diagram(problem_statement):
    prompt = generate_activity_diagram(problem_statement)
    response = llm.predict(prompt)
    return response

# Example usage
problem = problem_statement
activity_diagram = get_activity_diagram(problem)
print(activity_diagram)

C:\Users\admin\AppData\Local\Temp\ipykernel_1464\1354051585.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = rag_chain.run(problem_statement)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************zkoA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
def format_to_plantuml(activity_diagram_text):
    plantuml_code = "@startuml\nstart\n"

    steps = activity_diagram_text.split("\n")
    for step in steps:
        if "End" in step:
            plantuml_code += "stop\n"
        else:
            plantuml_code += f":{step.strip()};\n"

    plantuml_code += "@enduml"
    return plantuml_code

# Example
plantuml_code = format_to_plantuml(activity_diagram)
print(plantuml_code)


@startuml
start
:1. Start;
:2. Issue identity cards to employees / guest;
:3. Employee/Guest drives to car park entrance;
:4. Insert identity card into card reader;
:5. Verify card number;
:6. If card number recognized, trigger barrier to rise;
:7. If car park is not full, drive car into car park;
:8. If car park is full, display "Full" sign;
:9. When leaving, drive to car park exit;
:10. Exit barrier automatically raises;
:11. If card belongs to guest, return card to reception;
stop
@enduml
